<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [14]:
import numpy as np
import pandas as pd

import re

import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

from catboost import CatBoostClassifier
from sklearn.utils import class_weight

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [15]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [16]:
display(data.head(5))

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [18]:
display(data.duplicated().sum())

0

In [19]:
display(data.isna().sum())

Unnamed: 0    0
text          0
toxic         0
dtype: int64

Пропусков нет, явных дубликатов нет

## Обучение

In [20]:
data = data.drop(['Unnamed: 0'], axis=1)

In [21]:
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = ' '.join(text.split())
    return text

In [22]:
%%time
#очищаю тексты постов:
data['text'] = data['text'].apply(clear_text)

CPU times: user 4.59 s, sys: 71.4 ms, total: 4.67 s
Wall time: 4.68 s


In [23]:
data.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m se...,0
2,hey man i m really not trying to edit war it s...,0
3,more i can t make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0


In [33]:
lemmatizer = WordNetLemmatizer()
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [37]:
def lemm_text(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)

In [38]:
data['text'] = data['text'].apply(lemm_text)

In [39]:
features = data.drop(['toxic'], axis=1) 
target = data.toxic

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.2, 
                                                                              random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            test_size=0.5,
                                                                            random_state=12345)

In [40]:
for i in [features_train, target_train, features_valid, target_valid, features_test, target_test]:
    print(i.shape)


(127433, 1)
(127433,)
(15929, 1)
(15929,)
(15930, 1)
(15930,)


In [41]:
indices_1 = [i for i,x in enumerate(target_train) if x == 1]
count_1 = len(indices_1)

indices_0 = [i for i,x in enumerate(target_train) if x == 0]
count_0 = len(indices_0)

display('Доля значений 1 в тренировочной выборке:', len(indices_1) / (len(indices_1) + len(indices_0)))

'Доля значений 1 в тренировочной выборке:'

0.10166911239631807

In [42]:
toxic_train = data.iloc[target_train.index]
target_train_0 = toxic_train[toxic_train['toxic'] == 0]['toxic']
target_train_1 = toxic_train[toxic_train['toxic'] == 1]['toxic']


target_train_0_resample = target_train_0.sample(target_train_1.shape[0], random_state=12345)
target_train_resample = pd.concat([target_train_0_resample, target_train_1])

features_train_resample = data.iloc[target_train_resample.index]

features_train_resample, target_train_resample = shuffle(features_train_resample,
                                                         target_train_resample,
                                                         random_state=12345)

features_train_resample = features_train_resample.text 

display('Соотношение 1/0 в тренировочной выборке:')
display(target_train_resample.value_counts(normalize=True))

display(features_train_resample.shape)
display(target_train_resample.shape)

'Соотношение 1/0 в тренировочной выборке:'

0    0.5
1    0.5
Name: toxic, dtype: float64

(25912,)

(25912,)

Обучение

LogisticRegression

In [43]:
features_train = features_train.text

In [44]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english', sublinear_tf=True)), 
                     ("lr", LogisticRegression())])
    
parameters = {'lr__solver': ('liblinear', 'saga','newton-cg', 'lbfgs'),
              'lr__C': (.1, 1, 5, 10),
              'lr__random_state': ([12345]),
              'lr__max_iter': ([200]),
              'lr__class_weight': (['balanced'])}
                                            
gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

mts = gscv.cv_results_['mean_test_score']
lr_train_f1 = max(mts)

display('F1 логистической регрессии =', round(lr_train_f1,2))
display('при параметрах', gscv.best_params_)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not 

'F1 логистической регрессии ='

0.76

'при параметрах'

{'lr__C': 5,
 'lr__class_weight': 'balanced',
 'lr__max_iter': 200,
 'lr__random_state': 12345,
 'lr__solver': 'liblinear'}

CPU times: user 14min 51s, sys: 8min 14s, total: 23min 6s
Wall time: 23min 23s


In [45]:
#валидация:
predictions_valid = gscv.predict(features_valid.text)
lr_valid_f1 = f1_score(target_valid, predictions_valid)
display('F1 логистической регрессии на валидации =', round(lr_valid_f1,2))

'F1 логистической регрессии на валидации ='

0.76

In [46]:
#тестирование:
predictions_test = gscv.predict(features_test.text)
lr_test_f1 = f1_score(target_test, predictions_test)
display('финальный F1 логистической регрессии =', round(lr_test_f1,2))

'финальный F1 логистической регрессии ='

0.76

DecisionTreeClassifier

In [47]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("dtc", DecisionTreeClassifier())])
    
parameters = {'dtc__max_depth': ([x for x in range(1, 25)]),
              'dtc__random_state': ([12345]), 
              'dtc__class_weight': (['balanced'])}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train_resample, target_train_resample)

mts = gscv.cv_results_['mean_test_score']
dtc_train_f1 = max(mts)

display('F1 дерева решений =', round(dtc_train_f1,2))
display('при параметрах', gscv.best_params_)

'F1 дерева решений ='

0.71

'при параметрах'

{'dtc__class_weight': 'balanced',
 'dtc__max_depth': 24,
 'dtc__random_state': 12345}

CPU times: user 2min 6s, sys: 231 ms, total: 2min 6s
Wall time: 2min 6s


In [48]:
%%time

pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')),
    ("dtc", DecisionTreeClassifier())])
parameters = {
    'dtc__max_depth': [x for x in range(1, 25)],
    'dtc__random_state': [12345],
    'dtc__class_weight': ['balanced']}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)
gscv.fit(features_train, target_train)
dtc_train_f1 = max(gscv.cv_results_['mean_test_score'])
display('F1 дерева решений =', round(dtc_train_f1, 2))
display('при параметрах', gscv.best_params_)

'F1 дерева решений ='

0.63

'при параметрах'

{'dtc__class_weight': 'balanced',
 'dtc__max_depth': 24,
 'dtc__random_state': 12345}

CPU times: user 14min 17s, sys: 2.31 s, total: 14min 20s
Wall time: 14min 24s


In [49]:
#валидация:
predictions_valid = gscv.predict(features_valid.text)
dtc_valid_f1 = f1_score(target_valid, predictions_valid)
display('F1 дерева решений на валидации =', round(dtc_valid_f1,2))


'F1 дерева решений на валидации ='

0.63

In [50]:
#тестирование:
predictions_test = gscv.predict(features_test.text)
dtc_test_f1 = f1_score(target_test, predictions_test)
display('финальный F1 дерева решений =', round(dtc_test_f1,2))

'финальный F1 дерева решений ='

0.63

CatBoostClassifier

In [51]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("cbc", CatBoostClassifier())])
    
parameters = {'cbc__verbose': ([False]),
              'cbc__iterations': ([200])}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train_resample, target_train_resample)

mts = gscv.cv_results_['mean_test_score']
cbc_train_f1 = max(mts)

display('F1 CatBoostClassifier =', round(cbc_train_f1,2))
display('при параметрах', gscv.best_params_)

'F1 CatBoostClassifier ='

0.87

'при параметрах'

{'cbc__iterations': 200, 'cbc__verbose': False}

CPU times: user 6min 10s, sys: 1.95 s, total: 6min 12s
Wall time: 6min 22s


In [52]:
%%time

class_weights = class_weight.compute_class_weight('balanced', np.unique(target_train_resample), target_train_resample)
class_weights_dict = dict(enumerate(class_weights))

pipeline = Pipeline([
    ("vect", TfidfVectorizer(stop_words='english')),
    ("cbc", CatBoostClassifier(class_weights=class_weights_dict))
])
parameters = {
    'cbc__verbose': [False],
    'cbc__iterations': [200]
}
gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)
gscv.fit(features_train_resample, target_train_resample)
mts = gscv.cv_results_['mean_test_score']
cbc_train_f1 = max(mts)
display('F1 CatBoostClassifier =', round(cbc_train_f1,2))
display('при параметрах', gscv.best_params_)

/opt/conda/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1], y=12579     0
101766    1
81508     1
102215    0
150487    0
         ..
15357     0
69875     1
82464     0
150069    0
63672     1
Name: toxic, Length: 25912, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


'F1 CatBoostClassifier ='

0.87

'при параметрах'

{'cbc__iterations': 200, 'cbc__verbose': False}

CPU times: user 6min 12s, sys: 1.77 s, total: 6min 14s
Wall time: 6min 24s


In [53]:
#валидация:
predictions_valid = gscv.predict(features_valid.text)
cbc_valid_f1 = f1_score(target_valid, predictions_valid)
display('F1 CatBoostClassifier на валидации =', round(cbc_valid_f1,2))

'F1 CatBoostClassifier на валидации ='

0.72

In [55]:
#тестирование:
predictions_test = gscv.predict(features_test.text)
cbc_test_f1 = f1_score(target_test, predictions_test)
display('финальный F1 CatBoostClassifier =', round(cbc_test_f1,2))

'финальный F1 CatBoostClassifier ='

0.72

SGDClassifier

In [56]:
%%time

#обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')), 
                     ("clf", SGDClassifier())])
    
parameters = {'clf__loss': ('hinge', 'log', 'modified_huber'),
              'clf__learning_rate': ('constant', 'optimal', 'invscaling', 'adaptive'),
              'clf__eta0': (.01, .05, .1, .5),
              'clf__random_state': ([12345]),
              'clf__class_weight': (['balanced'])}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train_resample, target_train_resample)

mts = gscv.cv_results_['mean_test_score']
sgdc_train_f1 = max(mts)

display('F1 SGDClassifier =', round(sgdc_train_f1,2))
display('при параметрах', gscv.best_params_)

'F1 SGDClassifier ='

0.89

'при параметрах'

{'clf__class_weight': 'balanced',
 'clf__eta0': 0.01,
 'clf__learning_rate': 'adaptive',
 'clf__loss': 'modified_huber',
 'clf__random_state': 12345}

CPU times: user 3min 3s, sys: 11.5 s, total: 3min 15s
Wall time: 3min 17s


In [57]:
#валидация:
predictions_valid = gscv.predict(features_valid.text)
sgdc_valid_f1 = f1_score(target_valid, predictions_valid)
display('F1 SGDClassifier на валидации =', round(sgdc_valid_f1,2))

'F1 SGDClassifier на валидации ='

0.69

In [58]:
#тестирование:
predictions_test = gscv.predict(features_test.text)
sgdc_test_f1 = f1_score(target_test, predictions_test)
display('финальный F1 SGDClassifier =', round(sgdc_test_f1,2))

'финальный F1 SGDClassifier ='

0.7

## Выводы

In [59]:
index = ['LogisticRegression',
         'DecisionTreeClassifier',
         'CatBoostClassifier',
         'SGDClassifier'
        ]

data = {'F1 на обучающей выборке': [lr_train_f1,
                                    dtc_train_f1,
                                    cbc_train_f1,
                                    sgdc_train_f1],
        
        'F1 на валидационной выборке': [lr_valid_f1,
                                        dtc_valid_f1,
                                        cbc_valid_f1,
                                        sgdc_valid_f1],
        
        'F1 на тестовой выборке': [lr_test_f1,
                                               dtc_test_f1,
                                               cbc_test_f1,
                                               sgdc_test_f1]}

scores_data = pd.DataFrame(data=data, index=index)

scores_data.sort_values(by='F1 на тестовой выборке', ascending=False)

,F1 на обучающей выборке,F1 на валидационной выборке,F1 на тестовой выборке
LogisticRegression,0.755857,0.762490,0.763084
CatBoostClassifier,0.873915,0.719786,0.721583
SGDClassifier,0.891602,0.693450,0.697537
DecisionTreeClassifier,0.625409,0.631505,0.632056


Подготовленны данные обучения.

Выбран способ баланса классов, сформированы обучающая, валидационная и тестовая выборка.

Обучены модели и выбрана лучшая из них на валидационной выборке.

Показаны параметры качества моделей.

Исходные данные обладают большим количеством признаков. Созданных столбцов больше, чем записей данных. Лучшая модель стала LogisticRegression.

<div class="alert alert-block alert-success">
<b>Успех:</b> Приятно видеть вывод в конце проекта!
</div>

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны